In [33]:
import torch
import torch.nn as nn 
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [34]:
#STEP1
train_dataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset=dsets.MNIST(root='./data',
                         train=False,
                         transform=transforms.ToTensor())

In [3]:
print(train_dataset.train_data.size())

torch.Size([60000, 28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [4]:
print(train_dataset.train_labels.size())

torch.Size([60000])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:45: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [5]:
print(test_dataset.test_data.size())

torch.Size([10000, 28, 28])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [6]:
print(test_dataset.test_labels.size())

torch.Size([10000])


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [35]:
#STEP2
len(train_dataset)
batch_size=100
n_iters=3000
num_epochs= n_iters/(len(train_dataset)/batch_size)
num_epochs=int(num_epochs)
num_epochs
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [51]:
#STEP3
class CNNModel(nn.Module):
  def __init__(self):
     super(CNNModel, self).__init__()
     #convolution1
     self.cnn1= nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
     self.relu1= nn.ReLU()

     #MaxPool1
     self.maxpool1 = nn.MaxPool2d(kernel_size=2)

     #Convolution2
     self.cnn2= nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
     self.relu2 = nn.ReLU()

     #MaxPool2
     self.maxpool2 = nn.MaxPool2d(kernel_size=2)

     self.fc1= nn.Linear(32*7*7, 10)

  def forward(self, x):
    #Convolution 1
    out=self.cnn1(x) 
    out=self.relu1(out)

    #Max Pool 1
    out=self.maxpool1(out)
    
    #Convolution 2
    out= self.cnn2(out)
    out= self.relu2(out)
    
    #Max Pool 2
    out=self.maxpool2(out)

    #Resize
    #Original size: (100,32,7,7)
    #Out.size(0): 100
    #New out size: (100, 32*7*7)
    out= out.view(out.size(0), -1)

    #Linear Function
    out=self.fc1(out)

    return out

In [52]:
#STEP4
model= CNNModel()

In [53]:
#STEP5
criterion = nn.CrossEntropyLoss()

In [54]:
#STEP6
learning_rate=0.01

optimizer= torch.optim.SGD(model.parameters(), lr=learning_rate)

In [22]:
#parameters in depth
print(model.parameters())
print(len(list(model.parameters())))

#Convolution 1:16 kernels
print(list(model.parameters())[0].size())
#Convolution 1 Bias: 16 kernels
print(list(model.parameters())[1].size())

#Convolution 2:32 kernels with depth 16
print(list(model.parameters())[2].size())
#Convolution 2: 32 kernels with depth 16
print(list(model.parameters())[3].size())

#Fully Connected Layer 1
print(list(model.parameters())[4].size())
#Fully Connected Layer 1 with Bias
print(list(model.parameters())[5].size())


<generator object Module.parameters at 0x7f6bc61ceb48>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


In [55]:
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images
        images = images.requires_grad_()

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                # Load images
                images = images.requires_grad_()

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)

                # Total correct predictions
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


Iteration: 500. Loss: 0.6480942368507385. Accuracy: 84
Iteration: 1000. Loss: 0.3086514174938202. Accuracy: 92
Iteration: 1500. Loss: 0.17455345392227173. Accuracy: 94
Iteration: 2000. Loss: 0.19680136442184448. Accuracy: 95
Iteration: 2500. Loss: 0.23159915208816528. Accuracy: 96
Iteration: 3000. Loss: 0.1979455053806305. Accuracy: 96
